In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/LongRange3D/')
from models_binnedFeat import RandomForest, SVC
from metrics import ClassificationResult
from utils import concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale 

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN X (CNMeM is enabled with initial size: 2500 MB, cuDNN 5005)


In [2]:
contacts='PE'
path='/users/mtaranov/datasets_3d_OLD/dist_matched_'+contacts+'/'

X_train_node1 = path+'motifs/train_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/train_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_test_node1 = path+'motifs/test_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/test_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_valid_node1 = path+'motifs/valid_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/valid_'+contacts+'_out_node2_3_0.0001/mat.npy'

y_train = get_labels(path+'y_train_thres_10.npy')
y_test = get_labels(path+'y_test_thres_10.npy')
y_valid = get_labels(path+'y_valid_thres_10.npy')

X_train=concat_motifs([X_train_node1, X_train_node2])
X_test=concat_motifs([X_test_node1, X_test_node2])
X_valid=concat_motifs([X_valid_node1, X_valid_node2])

In [3]:
X_train.shape

(4824, 1, 1920, 2)

In [4]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features(X_train, X_valid, X_test)

In [5]:
X_train_pairs = X_train_normalized.reshape(X_train_normalized.shape[0],X_train_normalized.shape[2]*X_train_normalized.shape[3])
X_valid_pairs = X_valid_normalized.reshape(X_valid_normalized.shape[0],X_valid_normalized.shape[2]*X_valid_normalized.shape[3])
X_test_pairs = X_test_normalized.reshape(X_test_normalized.shape[0],X_test_normalized.shape[2]*X_test_normalized.shape[3])

In [6]:
X_train_pairs.shape

(4824, 3840)

# Random Forest

In [7]:
rf = RandomForest()

### with motifs only

In [8]:
rf.train(X_train_pairs, y_train)
preds_test = rf.predict(X_test_pairs)
preds_train = rf.predict(X_train_pairs)
print ('On Test:\n{}\n'.format(ClassificationResult(y_test, preds_test)))
print ('On Train:\n{}\n'.format(ClassificationResult(y_train, preds_train)))

/users/mtaranov/LongRange3D/models_binnedFeat.py:231: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.classifier.fit(X, y)


On Test:
Balanced Accuracy: 72.74%	 auROC: 0.811	 auPRC: 0.822	 auPRG: 0.644
Recall at 5%|10%|20% FDR: 16.8%|35.8%|62.6%	 Num Positives: 906	 Num Negatives: 906	 

On Train:
Balanced Accuracy: 100.00%	 auROC: 1.000	 auPRC: 1.000	 auPRG: 1.000
Recall at 5%|10%|20% FDR: 100.0%|100.0%|100.0%	 Num Positives: 2412	 Num Negatives: 2412	 



In [9]:
scores=rf.ImportanceSelect()

In [10]:
scores.shape

(3840,)

In [11]:
motif_names=np.loadtxt('/users/mtaranov/datasets_3d/dist_matched_PE/motifs/train_PE_out_btw_nodes_3_0.0001/motif_names.txt', dtype=str, delimiter='/')[:,6:].reshape(640,)
top_3_hits_names=[]
for i in motif_names:
    for j in range(3):
        top_3_hits_names.append(i)
top_3_hits_names = np.array(top_3_hits_names)

In [12]:
score_per_feature_node1=scores[0:3840:2]

In [13]:
score_per_feature_node2=scores[1:3841:2]

In [14]:
RF_Importance_Motifs_node1=top_3_hits_names[score_per_feature_node1.argsort()[::-1]]

In [15]:
RF_Importance_Motifs_node2=top_3_hits_names[score_per_feature_node2.argsort()[::-1]]

In [16]:
print np.where(RF_Importance_Motifs_node1=='CTCF_HUMAN')
print np.where(RF_Importance_Motifs_node1=='CTCFL_HUMAN')

(array([  0,  15, 980]),)
(array([  1,  11, 764]),)


In [19]:
print np.where(RF_Importance_Motifs_node2=='CTCF_HUMAN')
print np.where(RF_Importance_Motifs_node2=='CTCFL_HUMAN')

(array([133, 206, 840]),)
(array([ 595,  771, 1399]),)


In [17]:
RF_Importance_Motifs_node1[:100]

array(['CTCF_HUMAN', 'CTCFL_HUMAN', 'ONEC2_HUMAN', 'FOXG1_HUMAN',
       'FOXG1_HUMAN', 'IRF5_HUMAN', 'IRF8_HUMAN', 'ZN713_HUMAN',
       'NR2E3_HUMAN', 'THA_HUMAN', 'NR2E3_HUMAN', 'CTCFL_HUMAN',
       'BC11A_HUMAN', 'ONEC2_HUMAN', 'BARX2_HUMAN', 'CTCF_HUMAN',
       'ARI3A_HUMAN', 'ZN713_HUMAN', 'SP4_HUMAN', 'DLX1_HUMAN',
       'FOXL1_HUMAN', 'IRF1_HUMAN', 'ETV2_HUMAN', 'NFAT5_HUMAN',
       'FOXJ3_HUMAN', 'PRDM1_HUMAN', 'IRF7_HUMAN', 'IRF5_HUMAN',
       'FOXD2_HUMAN', 'PO4F3_HUMAN', 'ARI3A_HUMAN', 'IRF1_HUMAN',
       'FOXO1_HUMAN', 'NKX21_HUMAN', 'FOXJ3_HUMAN', 'DLX1_HUMAN',
       'BC11A_HUMAN', 'IRF5_HUMAN', 'EGR1_HUMAN', 'PO4F3_HUMAN',
       'FUBP1_HUMAN', 'PO3F3_HUMAN', 'OTX2_HUMAN', 'MAZ_HUMAN',
       'HIC2_HUMAN', 'THAP1_HUMAN', 'EHF_HUMAN', 'IRF4_HUMAN',
       'CPEB1_HUMAN', 'PLAG1_HUMAN', 'HXD9_HUMAN', 'FOXL1_HUMAN',
       'EGR1_HUMAN', 'PO3F3_HUMAN', 'SP2_HUMAN', 'MYOG_HUMAN',
       'SOX17_HUMAN', 'FOXG1_HUMAN', 'HXC10_HUMAN', 'IRF3_HUMAN',
       'FOXF1_HUMAN', 'MA

In [18]:
RF_Importance_Motifs_node2[:100]

array(['CPEB1_HUMAN', 'TBX2_HUMAN', 'HXC10_HUMAN', 'ARI3A_HUMAN',
       'HXC10_HUMAN', 'THA_HUMAN', 'EVX1_HUMAN', 'HXC6_HUMAN',
       'LMX1B_HUMAN', 'BC11A_HUMAN', 'MEF2D_HUMAN', 'TBX5_HUMAN',
       'PIT1_HUMAN', 'MAZ_HUMAN', 'NANOG_HUMAN', 'ZN713_HUMAN',
       'PO4F1_HUMAN', 'SOX15_HUMAN', 'MEF2A_HUMAN', 'HXC10_HUMAN',
       'ZN713_HUMAN', 'VDR_HUMAN', 'HNF1A_HUMAN', 'AP2B_HUMAN',
       'ONEC2_HUMAN', 'FOXG1_HUMAN', 'FOXL1_HUMAN', 'FOXB1_HUMAN',
       'FOXA2_HUMAN', 'FOXG1_HUMAN', 'RUNX2_HUMAN', 'CPEB1_HUMAN',
       'COE1_HUMAN', 'TFDP1_HUMAN', 'TFDP1_HUMAN', 'ETV2_HUMAN',
       'PRGR_HUMAN', 'THA_HUMAN', 'KLF16_HUMAN', 'BSH_HUMAN',
       'P5F1B_HUMAN', 'LMX1A_HUMAN', 'PIT1_HUMAN', 'PO3F4_HUMAN',
       'GSX1_HUMAN', 'TBX1_HUMAN', 'AIRE_HUMAN', 'COT2_HUMAN',
       'FOXA3_HUMAN', 'RFX5_HUMAN', 'FUBP1_HUMAN', 'PO3F3_HUMAN',
       'HMX1_HUMAN', 'SP4_HUMAN', 'THB_HUMAN', 'EVI1_HUMAN', 'ZN148_HUMAN',
       'FOXM1_HUMAN', 'THAP1_HUMAN', 'FOXO6_HUMAN', 'IRF8_HUMAN',
       'NR2E